In [1]:
import matplotlib.pyplot as plt
import numpy as np
from Nexullance_IT import Nexullance_IT
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from topologies.DDF import DDFtopo
import globals as gl
import copy
from joblib import Parallel, delayed
%matplotlib widget

# setup network, not yet using nexullance

In [2]:
config=(36, 5)
EPR=3
ddf_network = DDFtopo(config[0], config[1])
Cap_remote = 10 #GBps
Cap_local = 10 #GBps
M_EPs = gl.generate_half_shift_traffic_pattern(config[0], EPR) # an numpy matrix filled with ones
scaling_factor = 3
M_EPs = scaling_factor * M_EPs
M_R = gl.convert_M_EPs_to_M_R(M_EPs, config[0], EPR)

In [3]:
ASP = ddf_network.calculate_all_shortest_paths()
ECMP_ASP = gl.ECMP(ASP)
remote_link_flows, local_link_flows = ddf_network.distribute_M_EPs_on_weighted_paths(ECMP_ASP, EPR, M_EPs)
max_remote_link_load = np.max(remote_link_flows)/Cap_remote
max_local_link_load = np.max(local_link_flows)/Cap_local
print("Max remote link load: ", max_remote_link_load)
print("Max local link load: ", max_local_link_load)
if max_remote_link_load > 1.0 or max_local_link_load > 1.0:
    print("network congestion may present")
    mu = max_local_link_load/max_remote_link_load
    if mu > 1.0:
        print(f"mu = {mu} > 1, max local link load is higher")
    else:
        print(f"mu = {mu} < 1, max remote link load is higher")
else:
    print("network is probably not congested")
print("network_total_throughput = ", gl.network_total_throughput(M_EPs, max_remote_link_load, max_local_link_load), "GBps")

Max remote link load:  1.8
Max local link load:  0.3
network congestion may present
mu = 0.16666666666666666 < 1, max remote link load is higher
network_total_throughput =  180.0 GBps


In [4]:
alpha_1 = 40.0
beta_1 = 0.4

alpha_2 = 0
beta_2 = 7

def weighted_method_1(s: int, d: int, edge_attributes: dict):
    return alpha_1 + edge_attributes['load']**beta_1

def weighted_method_2(s: int, d: int, edge_attributes: dict):
    return alpha_2 + edge_attributes['load']**beta_2

In [5]:
nexu_it = Nexullance_IT(ddf_network.nx_graph, M_R, Cap_remote)

In [6]:
num_method_1 = 5
num_method_2 = 5
times_method_1, peakRAMs_method_1, time_method_2, peakRAM_method_2, results_method_1 = nexu_it.optimize_and_profile(num_method_1, num_method_2, weighted_method_1, weighted_method_2)

In [7]:
results_method_1

[1.8, 1.8, 2.25, 1.8, 2.25]

In [8]:
times_method_1

[0.4704113006591797,
 0.6685101985931396,
 0.6780295372009277,
 0.7575597763061523,
 0.684873104095459]

In [9]:
peakRAMs_method_1

[343054, 270840, 280888, 284344, 766784]

In [10]:
time_method_2

0.4429166316986084

In [11]:
peakRAM_method_2

244236

In [12]:
nexu_it.get_method_2_attempts()

50

In [13]:
nexu_it.get_result_max_link_load()

0.9

In [14]:
num_method_1 = 2
num_method_2 = 10
results_method_1, results_method_2 = nexu_it.optimize(num_method_1, num_method_2, weighted_method_1, weighted_method_2)

In [15]:
results_method_2

[[1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.8, 1.35, 1.35],
 [1.35,
  1.35,
  1.35,
  1.125,
  1.125,
  1.125,
  1.125,
  1.125,
  1.125,
  0.9000000000000001,
  1.125],
 [1.125,
  0.9000000000000001,
  0.9000000000000001,
  1.0125,
  1.0125,
  1.0125000000000002,
  1.0125,
  1.0125000000000002,
  1.0125,
  0.9000000000000001,
  0.9000000000000001],
 [0.9000000000000001,
  0.9000000000000001,
  0.9000000000000001,
  0.9562499999999999,
  0.9000000000000001,
  0.9000000000000001,
  0.95625,
  0.9000000000000002,
  0.9000000000000001,
  0.9000000000000001,
  0.9000000000000001],
 [0.9000000000000001,
  0.9000000000000001,
  0.9000000000000001,
  0.9,
  0.9,
  0.9,
  0.9,
  0.9,
  0.9,
  0.9,
  0.928125],
 [0.928125, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
 [0.9,
  0.9,
  0.9,
  0.9,
  0.9,
  0.8999999999999999,
  0.8999999999999999,
  0.8999999999999999,
  0.9000000000000001,
  0.90703125,
  0.9000000000000001],
 [0.9000000000000001,
  0.8999999999999999,
  0.892968750000

In [16]:
results_method_1 = np.array([gl.network_total_throughput(M_EPs, results_method_1[i], max_local_link_load) for i in range(len(results_method_1))])
results_method_2 = np.array([gl.network_total_throughput(M_EPs, results_method_2[i][-1], max_local_link_load) for i in range(len(results_method_2))])
OPTIMUM = 324.0  # Nexullance_OPT [GBps]

In [17]:
results_method_1/OPTIMUM

array([0.55555556, 0.55555556])

In [18]:
results_method_2/OPTIMUM

array([0.74074074, 0.88888889, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ])